In [11]:
from langchain_openai import ChatOpenAI

In [12]:
OPENAI_API_KEY = 

In [48]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY,model ='gpt-4o-mini', temperature=0)

In [49]:
response = llm.invoke("What is the live score of WI vs SA in today's match in the worldcup 2026")
print(response.content)

I'm sorry, but I can't provide live scores or real-time updates. You can check a sports news website, a sports app, or the official ICC website for the latest scores and updates on the match between West Indies and South Africa in the 2026 World Cup.


In [50]:
response = llm.invoke("Till what day of the data,you are trained")
print(response.content)

I am trained on data up until October 2021. If you have any questions or need information up to that date, feel free to ask!


In [51]:
response = llm.invoke("who was prime minister of india in 2014")
print(response.content)

In 2014, the Prime Minister of India was Narendra Modi. He took office on May 26, 2014, after his party, the Bharatiya Janata Party (BJP), won a majority in the general elections held that year.


In [52]:
response = llm.invoke("What is adress of lokesh mar")
print(response.content)

I'm sorry, but I can't assist with that.


In [53]:
response = llm.invoke("what are the class timings of innomatics ?")
print(response.content)

I'm sorry, but I don't have access to real-time information or specific details about institutions like Innomatics, including their class timings. I recommend checking their official website or contacting them directly for the most accurate and up-to-date information regarding class schedules.


# RAG System

## Context Creation

#### Data Loaders

In [14]:
from langchain_community.document_loaders import PyPDFLoader

In [18]:
loader = PyPDFLoader(r"C:\Users\User\Desktop\innomatics.pdf")
documents = loader.load()

In [20]:
import sys
sys.version

'3.13.11 | packaged by Anaconda, Inc. | (main, Dec 10 2025, 21:21:58) [MSC v.1929 64 bit (AMD64)]'

# Chunking

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [23]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)

In [25]:
chuked_data = splitter.split_documents(documents)

# Embeddings

In [27]:
from langchain_openai import OpenAIEmbeddings

In [35]:
openai_embedding = OpenAIEmbeddings(api_key = OPENAI_API_KEY)

## Vector Store

In [36]:
from langchain_chroma import Chroma

In [40]:
vectore_store = Chroma(embedding_function =openai_embedding,persist_directory = r'.\workshop' )

In [41]:
vectore_store.add_documents(chuked_data)

['2571a9bf-427f-4231-9b63-37c979d840d2',
 '3098887f-0527-4642-ae58-4b40e9f8dce0',
 '17045dd0-a4dd-481e-bc80-c9819f0ff049',
 'ef9758bd-c66d-4221-8c85-2f86e4982fa3',
 'c8940169-ac23-4202-a08e-c94de2444351',
 '1437dc52-d041-492d-ab99-616db87af7ce',
 'da897cf9-d215-4827-ac3d-f6f36de23b53',
 'e32ee0d0-e671-4a80-af0e-7826085c01ae',
 '3b3ecdfc-b78b-4fb4-81ac-42008511ac80']

#### GOOGLE GEMINI

In [34]:
#Embedding 
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [39]:
#gemini_embedding = GoogleGenerativeAIEmbeddings(api_key = )

SyntaxError: expected argument value expression (1409326183.py, line 1)

### 2. Retrieval

In [42]:
from langchain_core.prompts import ChatPromptTemplate

In [44]:
prompt = ChatPromptTemplate.from_template("""
Give the answerfor the question based on only context:
question:{question}
context:{context}

""")

In [45]:
retriever= vectore_store.as_retriever()

In [46]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [47]:
rag_chain = ({"context":retriever,"question":RunnablePassthrough()} |prompt | llm |StrOutputParser())

### 3. Generation

In [58]:
response = rag_chain.invoke("what are the class timings of innomatics ")
print(response)

The class timings for Innomatics are as follows:

- Offline: 9 AM - 11 AM and 11:30 AM - 1:30 PM
- Online: 7 AM - 9 AM and 8:30 PM - 10:30 PM


In [59]:
response = rag_chain.invoke("Tell me about the technical team of innomatics")
print(response)

The technical team at Innomatics Research Labs consists of industry-driven mentors with strong academic and practical expertise. They provide practical learning experiences, with each module handled by different industry experts. The program includes 2 hours of live instructor-led sessions and 4 hours of mentoring support daily. Innomatics has close to 70 industry experts involved in delivering the program, ensuring comprehensive coverage of data science topics. Additionally, they offer over 200 use cases and 5-6 hands-on projects for participants to work on during the course.


In [61]:
response = rag_chain.invoke("who are the trainers")
print(response)

The trainers are:

- Mr. Nagaraju Ekkirala – Lead Data Scientist with 8+ years of teaching experience.
- Ms. Tasleema Noor – Experienced Data Scientist with extensive teaching expertise.
- Mr. Manoj Gaikwad – Industry professional with strong hands-on knowledge.
- Mr. Lohith – Experienced trainer with a solid background in EdTech platforms.


In [62]:
response = rag_chain.invoke("what are the contact numbers to register for the course ?")
print(response)

The contact numbers to register for the course are:
- 7396282424 – Karteek Mundru
- 7396127171 – Vineetha
- 099516 66674


In [63]:
response = rag_chain.invoke("what is the adress of innomatics ?")
print(response)

The address of Innomatics is:

2nd Floor, Innomatics Research Labs  
Plot No. 109, Beside Victoria Memorial Metro Station  
Saroornagar, Narsimha Puri Colony, HUDA Colony  
Kothapet, Hyderabad – 500035
